In [1]:
#TO DO


#did not normalize or standardize

#nans in test set to 180

# Do I want to make the padding value 0 so i ignore all rows with 0??
# should i use a different optimizer for my compiler?? maybe rmsprop

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
import pandas as pd
import os
import csv
import numpy as np
from tensorflow.keras import Sequential, layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
ROOT_DATA_DIR = '/content/drive/MyDrive/Squat_Pro/Out_Feature_CSVs'
#ROOT_DATA_DIR = '../raw_data/Out_Feature_CSVs'
DATA_SUB_DIRs = ["Train", "Test"]
CLS_LIST = ["Bad", "Good"]

def get_y(data_path, class_list):
    
    data_dict = {"csv_paths":[], "csv_files":[], "labels":[]}
    
    for i, clss in enumerate(class_list):
        dir_path = os.path.join(data_path, clss)
        list_csvs = [l for l in os.listdir(dir_path)]
        print(f"Found {len(list_csvs)} CSVs in {dir_path} Directory")
        data_dict["csv_files"].extend(list_csvs)
        data_dict["labels"].extend([i for k in range(len(list_csvs))])
        data_dict["csv_paths"].extend([dir_path for _ in range(len(list_csvs))])
    
    total_csvs = len(data_dict["csv_files"])
    total_labels = len(data_dict["labels"])
    print(f"Number of csvs : {total_csvs}")
    print(f"Number of Labels : {total_labels}")
    return data_dict

train_data = pd.DataFrame(get_y(data_path=os.path.join(ROOT_DATA_DIR, DATA_SUB_DIRs[0]),
                               class_list=CLS_LIST))


test_data = pd.DataFrame(get_y(data_path=os.path.join(ROOT_DATA_DIR, DATA_SUB_DIRs[1]),
                               class_list=CLS_LIST))  

Found 40 CSVs in /content/drive/MyDrive/Squat_Pro/Out_Feature_CSVs/Train/Bad Directory
Found 40 CSVs in /content/drive/MyDrive/Squat_Pro/Out_Feature_CSVs/Train/Good Directory
Number of csvs : 80
Number of Labels : 80
Found 10 CSVs in /content/drive/MyDrive/Squat_Pro/Out_Feature_CSVs/Test/Bad Directory
Found 10 CSVs in /content/drive/MyDrive/Squat_Pro/Out_Feature_CSVs/Test/Good Directory
Number of csvs : 20
Number of Labels : 20


In [5]:
train_data.head()

,csv_paths,csv_files,labels
0,/content/drive/MyDrive/Squat_Pro/Out_Feature_C...,74.csv,0
1,/content/drive/MyDrive/Squat_Pro/Out_Feature_C...,53.csv,0
2,/content/drive/MyDrive/Squat_Pro/Out_Feature_C...,70.csv,0
3,/content/drive/MyDrive/Squat_Pro/Out_Feature_C...,62.csv,0
4,/content/drive/MyDrive/Squat_Pro/Out_Feature_C...,65.csv,0


In [6]:
train_data['csv_paths'][0]

'/content/drive/MyDrive/Squat_Pro/Out_Feature_CSVs/Train/Bad'

In [7]:
train_data.tail()

,csv_paths,csv_files,labels
75,/content/drive/MyDrive/Squat_Pro/Out_Feature_C...,9.csv,1
76,/content/drive/MyDrive/Squat_Pro/Out_Feature_C...,17.csv,1
77,/content/drive/MyDrive/Squat_Pro/Out_Feature_C...,5.csv,1
78,/content/drive/MyDrive/Squat_Pro/Out_Feature_C...,12.csv,1
79,/content/drive/MyDrive/Squat_Pro/Out_Feature_C...,40.csv,1


In [8]:
y_train = train_data['labels']

In [9]:
y_train.shape

(80,)

In [10]:
y_test = test_data['labels']

In [11]:
y_test.shape

(20,)

In [12]:
df = pd.read_csv('/content/drive/MyDrive/Squat_Pro/Out_Feature_CSVs/Train/Bad/76.csv')
df.iloc[:,1:]

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,...,feature_2008,feature_2009,feature_2010,feature_2011,feature_2012,feature_2013,feature_2014,feature_2015,feature_2016,feature_2017,feature_2018,feature_2019,feature_2020,feature_2021,feature_2022,feature_2023,feature_2024,feature_2025,feature_2026,feature_2027,feature_2028,feature_2029,feature_2030,feature_2031,feature_2032,feature_2033,feature_2034,feature_2035,feature_2036,feature_2037,feature_2038,feature_2039,feature_2040,feature_2041,feature_2042,feature_2043,feature_2044,feature_2045,feature_2046,feature_2047
0,0.246979,0.843562,0.071968,0.124413,0.999673,0.076203,0.108685,0.965372,2.013730,1.317685,0.722400,0.312725,0.160427,0.017920,0.321274,0.463303,0.081900,0.205143,0.055495,0.530935,0.254383,0.150312,0.018832,0.225992,0.191963,1.275720,0.348358,0.015737,0.004457,0.000624,0.212152,0.437991,0.029661,0.141176,0.236344,0.288691,0.274029,1.157004,0.734003,0.572620,...,0.227513,0.087194,0.249747,0.228253,0.000806,0.466527,0.813007,0.108993,0.053977,0.367273,0.208522,0.135577,0.800517,0.358892,0.105964,0.0,0.643910,1.071340,0.074552,0.607349,0.008821,0.308869,0.447797,0.178835,0.643049,0.270678,0.256715,0.174197,0.246554,0.932660,0.023789,0.408551,1.216399,0.726352,0.114373,0.163468,0.526327,0.188219,0.061620,0.754256
1,0.259899,0.924879,0.050829,0.138344,1.005638,0.037128,0.135432,0.918760,2.127180,1.064939,0.409487,0.125304,0.117467,0.011530,0.378844,0.544974,0.220873,0.084282,0.144265,0.436793,0.168110,0.272019,0.009891,0.325496,0.123113,1.014257,0.315338,0.037318,0.014505,0.013988,0.169956,0.369916,0.054119,0.120136,0.261728,0.214453,0.138480,1.190073,0.599680,0.740078,...,0.329329,0.063473,0.084560,0.202571,0.001420,0.374988,0.675173,0.092619,0.088150,0.446891,0.342444,0.215340,0.877520,0.464901,0.086024,0.0,0.629395,1.113403,0.049020,0.722569,0.002074,0.340971,0.549071,0.151153,0.470810,0.426477,0.332978,0.168887,0.228578,0.975402,0.023161,0.361161,1.214534,0.646354,0.113299,0.111966,0.553718,0.155690,0.119376,0.724960
2,0.383684,1.082262,0.008478,0.108694,0.948020,0.055017,0.122317,0.945775,2.014079,1.146610,0.456787,0.207925,0.024979,0.011684,0.415860,0.561660,0.226034,0.064355,0.136073,0.350083,0.125251,0.324499,0.009309,0.300237,0.131508,0.958490,0.253908,0.001762,0.000000,0.019737,0.158943,0.396542,0.045946,0.082320,0.320263,0.263122,0.099719,1.203329,0.564605,0.855648,...,0.380146,0.060137,0.142886,0.223425,0.012137,0.392088,0.771830,0.047923,0.013133,0.473170,0.163712,0.212929,0.890232,0.427094,0.079549,0.0,0.595238,0.959501,0.031393,0.653636,0.001628,0.339187,0.494618,0.147194,0.485451,0.630060,0.249511,0.154219,0.236492,0.981614,0.013676,0.239370,1.234755,0.613653,0.114273,0.079649,0.711490,0.227936,0.072618,0.738956
3,0.333121,1.032829,0.006871,0.066413,0.872104,0.063404,0.143408,1.065072,2.022523,1.078078,0.281922,0.103308,0.160011,0.008638,0.481407,0.655089,0.278022,0.133802,0.122334,0.295182,0.136236,0.283220,0.000000,0.215845,0.143795,0.774676,0.261971,0.000000,0.000000,0.000012,0.170434,0.299338,0.131817,0.069371,0.243737,0.265417,0.037839,1.396533,0.445555,0.757898,...,0.590719,0.049420,0.081046,0.162542,0.003681,0.361090,0.637323,0.031488,0.058747,0.555180,0.180691,0.279776,1.541735,0.333137,0.100323,0.0,0.675823,1.024275,0.021318,0.729329,0.002936,0.256907,0.470671,0.122232,0.386787,0.427113,0.110523,0.121584,0.311919,0.915962,0.014408,0.239992,1.209389,0.518542,0.246874,0.054760,0.672378,0.211177,0.033930,0.654003
4,0.453533,0.960070,0.006348,0.061298,1.082252,0.063146,0.083700,0.886553,1.957184,1.048781,0.335322,0.119818,0.112907,0.001420,0.480443,0.780236,0.273630,0.2279

In [13]:
def csv_to_list_of_lists(csv_path):
    df = pd.read_csv(csv_path)
    df.fillna(180, inplace=True)
    list_of_lists = df.iloc[:,1:].values.tolist()
    return list_of_lists

In [14]:
def get_x(dataframe):
  list_of_csv_files = [os.path.join(row['csv_paths'], row["csv_files"]) for index, row in dataframe.iterrows()]
  X_list = [csv_to_list_of_lists(f) for f in list_of_csv_files]
  return X_list

In [15]:
#X_train is a list of lists of different list lengths
X_train = get_x(train_data)

In [16]:
#pad X_train so all arrays are of the same shape
X_train_pad = pad_sequences(X_train, dtype='float32',padding='post',value=-1000)
X_train_pad.shape

(80, 176, 2048)

In [17]:
#X_test is a list of lists of different list lengths
X_test = get_x(test_data)

In [18]:
#pad X_test so all arrays are of the same shape
X_test_pad = pad_sequences(X_test, dtype='float32',padding='post',value=-1000)
X_test_pad.shape

(20, 135, 2048)

#LSTM RNN Model_1 (.55 - .60 accuracy)
model_LSTM = Sequential()

##change masking layer to padded amount
model_LSTM.add(layers.Masking(mask_value=-1000))

#how many units should the first layer be??
model_LSTM.add(layers.LSTM(20, activation='tanh'))  
model_LSTM.add(layers.Dense(10, activation='relu'))
model_LSTM.add(layers.Dense(1, activation='sigmoid'))

model_LSTM.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [24]:
#LSTM RNN Model_2 (.70 - accuracy)
model_LSTM = Sequential()

##change masking layer to padded amount
model_LSTM.add(layers.Masking(mask_value=-1000))

#how many units should the first layer be??
model_LSTM.add(layers.LSTM(200, activation='tanh'))
model_LSTM.add(layers.Dense(125, activation='relu'))
model_LSTM.add(layers.Dense(100, activation='relu'))
model_LSTM.add(layers.Dense(75, activation='relu'))
model_LSTM.add(layers.Dense(60, activation='relu'))
model_LSTM.add(layers.Dense(40, activation='relu'))
model_LSTM.add(layers.Dense(25, activation='relu'))    
model_LSTM.add(layers.Dense(10, activation='relu'))
model_LSTM.add(layers.Dense(1, activation='sigmoid'))

model_LSTM.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

##Hogan's first layer (do i need to add an input dimension??)
model.add(layers.SimpleRNN(500, input_dim=1, activation='relu'))

#how many units should the first layer be??
model.add(layers.LSTM(units=500, activation='tanh'))  
model.add(layers.Dense(250, activation='relu'))
model.add(layers.Dense(125, activation='relu'))
model.add(layers.Dense(75, activation='relu'))
model.add(layers.Dense(25, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [25]:
#Fitting the model to the train set
es = EarlyStopping(patience=5)

model_LSTM.fit(X_train_pad, y_train, 
              epochs=20, 
              batch_size=32, 
              verbose=1, 
              callbacks = [es],
              validation_split=0.2)

Epoch 1/20
2/2 [==============================] - 10s 5s/step - loss: 0.6827 - accuracy: 0.6354 - val_loss: 0.9360 - val_accuracy: 0.0000e+00
Epoch 2/20
2/2 [==============================] - 4s 2s/step - loss: 0.6331 - accuracy: 0.6458 - val_loss: 1.1469 - val_accuracy: 0.0000e+00
Epoch 3/20
2/2 [==============================] - 3s 2s/step - loss: 0.5872 - accuracy: 0.6875 - val_loss: 1.2355 - val_accuracy: 0.1250
Epoch 4/20
2/2 [==============================] - 3s 2s/step - loss: 0.5618 - accuracy: 0.7604 - val_loss: 1.2927 - val_accuracy: 0.1250
Epoch 5/20
2/2 [==============================] - 3s 2s/step - loss: 0.5394 - accuracy: 0.7812 - val_loss: 1.3128 - val_accuracy: 0.1250
Epoch 6/20
2/2 [==============================] - 3s 2s/step - loss: 0.5258 - accuracy: 0.7604 - val_loss: 1.2745 - val_accuracy: 0.1250


In [21]:
#Evaluate the model on the test set
model_LSTM.evaluate(X_test_pad, y_test)

1/1 [==============================] - 3s 3s/step - loss: 0.6441 - accuracy: 0.7000


[0.64411461353302, 0.699999988079071]